In [1]:
import forecast_tools as ft
import numpy as np
from numpy import sqrt, mean, square
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, LSTM, Embedding, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import  EarlyStopping, ModelCheckpoint, TensorBoard, \
                                        ReduceLROnPlateau


pd.options.plotting.backend = "plotly"
pd.set_option('precision', 2)

2023-01-16 02:03:42.391511: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-16 02:03:42.395749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 02:03:42.395761: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def one_hot_of_peaks(ds,freq='D'):
  df = pd.DataFrame(ds)
  df['peak'] = 0
  df.loc[df.groupby(pd.Grouper(freq=freq)).idxmax().iloc[:,0], 'peak'] = 1  
  return df['peak']   

def accuracy_one_hot(true,pred):
    """ Measure the accuracy of two one hot vectors, inputs can be 1d numpy or dataseries"""
    n_misses = sum(true != pred)/2     # every miss gives two 'False' entries
    return 1 - n_misses/sum(true)   # basis is the number of one-hots

# gru

In [3]:
results = {}

for sequence_length in [12,24,48,24*7]:
    for units in [24,36,48,64,72,128,256]:
        site='prpa'
        epochs=300
        dropout=0
        patience=100
        layers=1
        verbose=0
        output = True
        plots = False
        filename = 'data/PRPA_load_cleaned_mjw.csv'
        shift_steps = 1
        dir = 'models'
        features = [  'Load (kW)',
                    'Day',
                    'Weekday',
                    'Hour',
                    'IMF1',                                
                    'IMF2',                                
                    'IMF3',
                    'IMF4',
                    'IMF5',
                    'IMF6',
                    'IMF7',
                    'IMF8',]
        targets = ['TargetsOH']
        train_split = 0.9
        afuncs={'lstm':'relu','dense':'relu','gru':'relu'}
        loss='binary_crossentropy'
        metrics=['accuracy']

        t = datetime.now()
        path_checkpoint = f'{dir}/{site}/{t.year}-{t.month:02}-{t.day:02}_'+\
                        f'{t.hour:02}-{t.minute:02}-{t.second:02}_lstm_{units}x{layers}x{shift_steps}.keras'

        df,dppd,np_days = ft.get_dat_v4(site,filename,emd=True,rename=True,start='2021-6-7',end='2022-8-10')                    

        df['LoadOH'] =      one_hot_of_peaks(df[['Load (kW)']])
        df['TargetsOH'] =   one_hot_of_peaks(df[['Load (kW)']]).shift(-shift_steps)
        df['PredNPOH'] =    one_hot_of_peaks(df[['Load (kW)']]).shift(np_days*dppd-shift_steps)
        df = df.dropna()
        
        # split
        num_data = len(df)
        num_train = int(train_split * num_data)
        df_train = df.iloc[:num_train,:]
        df_valid = df.iloc[num_train:,:]

        feature_scaler = MinMaxScaler()
        X_train = feature_scaler.fit_transform(df_train[features].values)
        X_valid = feature_scaler.fit_transform(df_valid[features].values)

        y_train = df_train.TargetsOH.values[:,np.newaxis]
        y_valid = df_valid.TargetsOH.values[:,np.newaxis]
        
        generator = ft.batch_generator( batch_size=32,
                                        sequence_length=sequence_length,
                                        num_x_signals=len(features),
                                        num_y_signals=len(targets),
                                        num_train=num_train,
                                        x_train_scaled=X_train,
                                        y_train_scaled=y_train)   
        
        X_batch, y_batch = next(generator)
        
        X_valid = X_valid[np.newaxis,:,:]
        y_valid = y_valid[np.newaxis,:,:] 

        model = Sequential()
        model.add( GRU(  units=units,
                        return_sequences=True,
                        input_shape=(None, len(features),),
                        activation=afuncs['gru']))
        model.add(Dense(len(targets), activation=afuncs['dense']))

        model.compile(loss=loss, optimizer='adam',metrics=metrics)
        model.summary()                  

        callback_checkpoint = ModelCheckpoint(  filepath=path_checkpoint,
                                                monitor='val_loss',
                                                verbose=verbose,
                                                save_weights_only=True,
                                                save_best_only=True)

        callback_early_stopping = EarlyStopping(  monitor='val_loss',
                                                patience=patience,
                                                verbose=verbose)

        callbacks = [ callback_early_stopping,
                    callback_checkpoint,]

        hx = model.fit(  x=generator,
                    epochs=100,
                    steps_per_epoch=100,
                    validation_data=(X_valid,y_valid),
                    callbacks=callbacks)        
        
        model.load_weights(path_checkpoint)
        y_valid_pred = model.predict(X_valid)
        
        y_valid_flat      = y_valid[:,:,0].flatten()
        y_valid_pred_flat = y_valid_pred[:,:,0].flatten()
        
        df_valid.loc[:,'y'] = y_valid_flat
        df_valid.loc[:,'y_pred'] = y_valid_pred_flat    
        
        results[f'u{units} sl{sequence_length}'] = accuracy_one_hot(df_valid['y'],one_hot_of_peaks(df_valid['y_pred']))
        

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 24)          2736      
                                                                 
 dense (Dense)               (None, None, 1)           25        
                                                                 
Total params: 2,761
Trainable params: 2,761
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


2023-01-16 02:03:50.972150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-16 02:03:50.972169: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-16 02:03:50.972182: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (atium): /proc/driver/nvidia/version does not exist
2023-01-16 02:03:50.972337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


100/100 [==============================] - 2s 7ms/step - loss: 0.2932 - accuracy: 0.9588 - val_loss: 0.2210 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 0s 3ms/step - loss: 0.2335 - accuracy: 0.9578 - val_loss: 0.1980 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 0s 4ms/step - loss: 0.1839 - accuracy: 0.9585 - val_loss: 0.1711 - val_accuracy: 0.9583
Epoch 4/100
100/100 [==============================] - 0s 4ms/step - loss: 0.1714 - accuracy: 0.9582 - val_loss: 0.1562 - val_accuracy: 0.9583
Epoch 5/100
100/100 [==============================] - 0s 3ms/step - loss: 0.1659 - accuracy: 0.9576 - val_loss: 0.1559 - val_accuracy: 0.9583
Epoch 6/100
100/100 [==============================] - 0s 3ms/step - loss: 0.1847 - accuracy: 0.9592 - val_loss: 0.1671 - val_accuracy: 0.9583
Epoch 7/100
100/100 [==============================] - 0s 3ms/step - loss: 0.1545 - accuracy: 0.9585 - val_loss: 0.1595 - val_accuracy: 0.9583
Epoch 8/100

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, None, 36)          5400      
                                                                 
 dense_1 (Dense)             (None, None, 1)           37        
                                                                 
Total params: 5,437
Trainable params: 5,437
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 1s 5ms/step - loss: 0.2433 - accuracy: 0.9580 - val_loss: 0.2838 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 0s 4ms/step - loss: 0.1917 - accuracy: 0.9592 - val_loss: 0.1919 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 0s 4ms/step - loss: 0.1734 - accuracy: 0.9581 - val_loss: 0.1715 - val_accuracy: 0.9583
Epoch 4/100
100/100

...

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_26 (GRU)                (None, None, 128)         54528     
                                                                 
 dense_26 (Dense)            (None, None, 1)           129       
                                                                 
Total params: 54,657
Trainable params: 54,657
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 8s 65ms/step - loss: 0.1536 - accuracy: 0.9583 - val_loss: 0.1239 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 6s 63ms/step - loss: 0.1300 - accuracy: 0.9584 - val_loss: 0.1311 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 6s 63ms/step - loss: 0.1163 - accuracy: 0.9583 - val_loss: 0.1643 - val_accuracy: 0.9583
Epoch 4/100
1

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_27 (GRU)                (None, None, 256)         207360    
                                                                 
 dense_27 (Dense)            (None, None, 1)           257       
                                                                 
Total params: 207,617
Trainable params: 207,617
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 14s 120ms/step - loss: 0.1526 - accuracy: 0.9584 - val_loss: 0.1199 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 12s 119ms/step - loss: 0.1295 - accuracy: 0.9583 - val_loss: 0.1846 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 12s 117ms/step - loss: 0.1455 - accuracy: 0.9583 - val_loss: 0.1226 - val_accuracy: 0.9583
Epoch

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [4]:
results

{'u24 sl12': 0.2325581395348837,
 'u36 sl12': 0.18604651162790697,
 'u48 sl12': 0.18604651162790697,
 'u64 sl12': 0.2558139534883721,
 'u72 sl12': 0.3023255813953488,
 'u128 sl12': 0.2790697674418605,
 'u256 sl12': 0.2558139534883721,
 'u24 sl24': 0.2790697674418605,
 'u36 sl24': 0.32558139534883723,
 'u48 sl24': 0.2558139534883721,
 'u64 sl24': 0.2325581395348837,
 'u72 sl24': 0.34883720930232553,
 'u128 sl24': 0.2093023255813954,
 'u256 sl24': 0.2325581395348837,
 'u24 sl48': 0.2790697674418605,
 'u36 sl48': 0.18604651162790697,
 'u48 sl48': 0.18604651162790697,
 'u64 sl48': 0.3023255813953488,
 'u72 sl48': 0.0,
 'u128 sl48': 0.0,
 'u256 sl48': 0.2325581395348837,
 'u24 sl168': 0.2093023255813954,
 'u36 sl168': 0.2325581395348837,
 'u48 sl168': 0.2093023255813954,
 'u64 sl168': 0.2325581395348837,
 'u72 sl168': 0.0,
 'u128 sl168': 0.3023255813953488,
 'u256 sl168': 0.2558139534883721}

# lstm

## 1 layer

In [5]:
results = {}

for sequence_length in [12,24,48,24*7]:
    for units in [24,36,48,64,72,128,256]:
        site='prpa'
        epochs=300
        dropout=0
        patience=100
        layers=1
        verbose=0
        output = True
        plots = False
        filename = 'data/PRPA_load_cleaned_mjw.csv'
        shift_steps = 1
        dir = 'models'
        features = [  'Load (kW)',
                    'Day',
                    'Weekday',
                    'Hour',
                    'IMF1',                                
                    'IMF2',                                
                    'IMF3',
                    'IMF4',
                    'IMF5',
                    'IMF6',
                    'IMF7',
                    'IMF8',]
        targets = ['TargetsOH']
        train_split = 0.9
        afuncs={'lstm':'relu','dense':'relu','gru':'relu'}
        loss='binary_crossentropy'
        metrics=['accuracy']

        t = datetime.now()
        path_checkpoint = f'{dir}/{site}/{t.year}-{t.month:02}-{t.day:02}_'+\
                        f'{t.hour:02}-{t.minute:02}-{t.second:02}_lstm_{units}x{layers}x{shift_steps}.keras'

        df,dppd,np_days = ft.get_dat_v4(site,filename,emd=True,rename=True,start='2021-6-7',end='2022-8-10')                    

        df['LoadOH'] =      one_hot_of_peaks(df[['Load (kW)']])
        df['TargetsOH'] =   one_hot_of_peaks(df[['Load (kW)']]).shift(-shift_steps)
        df['PredNPOH'] =    one_hot_of_peaks(df[['Load (kW)']]).shift(np_days*dppd-shift_steps)
        df = df.dropna()
        
        # split
        num_data = len(df)
        num_train = int(train_split * num_data)
        df_train = df.iloc[:num_train,:]
        df_valid = df.iloc[num_train:,:]

        feature_scaler = MinMaxScaler()
        X_train = feature_scaler.fit_transform(df_train[features].values)
        X_valid = feature_scaler.fit_transform(df_valid[features].values)

        y_train = df_train.TargetsOH.values[:,np.newaxis]
        y_valid = df_valid.TargetsOH.values[:,np.newaxis]
        
        generator = ft.batch_generator( batch_size=32,
                                        sequence_length=sequence_length,
                                        num_x_signals=len(features),
                                        num_y_signals=len(targets),
                                        num_train=num_train,
                                        x_train_scaled=X_train,
                                        y_train_scaled=y_train)   
        
        X_batch, y_batch = next(generator)
        
        X_valid = X_valid[np.newaxis,:,:]
        y_valid = y_valid[np.newaxis,:,:] 

        model = Sequential()
        model.add( LSTM(  units=units,
                        return_sequences=True,
                        input_shape=(None, len(features),),
                        activation=afuncs['lstm']))
        model.add(Dense(len(targets), activation=afuncs['dense']))

        model.compile(loss=loss, optimizer='adam',metrics=metrics)
        model.summary()                  

        callback_checkpoint = ModelCheckpoint(  filepath=path_checkpoint,
                                                monitor='val_loss',
                                                verbose=verbose,
                                                save_weights_only=True,
                                                save_best_only=True)

        callback_early_stopping = EarlyStopping(  monitor='val_loss',
                                                patience=patience,
                                                verbose=verbose)

        callbacks = [ callback_early_stopping,
                    callback_checkpoint,]

        hx = model.fit(  x=generator,
                    epochs=100,
                    steps_per_epoch=100,
                    validation_data=(X_valid,y_valid),
                    callbacks=callbacks)        
        
        model.load_weights(path_checkpoint)
        y_valid_pred = model.predict(X_valid)
        
        y_valid_flat      = y_valid[:,:,0].flatten()
        y_valid_pred_flat = y_valid_pred[:,:,0].flatten()
        
        df_valid.loc[:,'y'] = y_valid_flat
        df_valid.loc[:,'y_pred'] = y_valid_pred_flat    
        
        results[f'u{units} sl{sequence_length}'] = accuracy_one_hot(df_valid['y'],one_hot_of_peaks(df_valid['y_pred']))
        

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 24)          3552      
                                                                 
 dense_28 (Dense)            (None, None, 1)           25        
                                                                 
Total params: 3,577
Trainable params: 3,577
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 1s 6ms/step - loss: 0.3455 - accuracy: 0.9577 - val_loss: 0.3455 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 0s 3ms/step - loss: 0.1902 - accuracy: 0.9576 - val_loss: 0.2539 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 0s 4ms/step - loss: 0.1593 - accuracy: 0.9590 - val_loss: 0.1647 - val_accuracy: 0.9583
Epoch 4/100
100/10

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, None, 36)          7056      
                                                                 
 dense_29 (Dense)            (None, None, 1)           37        
                                                                 
Total params: 7,093
Trainable params: 7,093
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 1s 6ms/step - loss: 0.2053 - accuracy: 0.9565 - val_loss: 0.3265 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 0s 4ms/step - loss: 0.1699 - accuracy: 0.9594 - val_loss: 0.1844 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 0s 4ms/step - loss: 0.1587 - accuracy: 0.9578 - val_loss: 0.1495 - val_accuracy: 0.9583
Epoch 4/100
100/10

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, None, 48)          11712     
                                                                 
 dense_30 (Dense)            (None, None, 1)           49        
                                                                 
Total params: 11,761
Trainable params: 11,761
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 1s 6ms/step - loss: 0.1945 - accuracy: 0.9576 - val_loss: 0.4262 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 0s 5ms/step - loss: 0.2204 - accuracy: 0.9583 - val_loss: 0.3429 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 0s 4ms/step - loss: 0.1863 - accuracy: 0.9577 - val_loss: 0.3797 - val_accuracy: 0.9583
Epoch 4/100
100/

...

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_26 (LSTM)              (None, None, 128)         72192     
                                                                 
 dense_54 (Dense)            (None, None, 1)           129       
                                                                 
Total params: 72,321
Trainable params: 72,321
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 7s 68ms/step - loss: 0.1575 - accuracy: 0.9583 - val_loss: 0.1236 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 7s 66ms/step - loss: 0.1362 - accuracy: 0.9583 - val_loss: 0.1190 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 7s 67ms/step - loss: 0.1365 - accuracy: 0.9583 - val_loss: 0.1275 - val_accuracy: 0.9583
Epoch 4/100
1

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_27 (LSTM)              (None, None, 256)         275456    
                                                                 
 dense_55 (Dense)            (None, None, 1)           257       
                                                                 
Total params: 275,713
Trainable params: 275,713
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 15s 140ms/step - loss: 0.1612 - accuracy: 0.9583 - val_loss: 0.1248 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 13s 135ms/step - loss: 0.1392 - accuracy: 0.9583 - val_loss: 0.1416 - val_accuracy: 0.9583
Epoch 3/100
100/100 [==============================] - 14s 142ms/step - loss: 0.1347 - accuracy: 0.9583 - val_loss: 0.1161 - val_accuracy: 0.9583
Epoch

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [6]:
results

{'u24 sl12': 0.023255813953488413,
 'u36 sl12': 0.2325581395348837,
 'u48 sl12': 0.2325581395348837,
 'u64 sl12': 0.3023255813953488,
 'u72 sl12': 0.0,
 'u128 sl12': 0.0,
 'u256 sl12': 0.0,
 'u24 sl24': 0.16279069767441856,
 'u36 sl24': 0.2093023255813954,
 'u48 sl24': 0.2558139534883721,
 'u64 sl24': 0.13953488372093026,
 'u72 sl24': 0.18604651162790697,
 'u128 sl24': 0.2325581395348837,
 'u256 sl24': 0.16279069767441856,
 'u24 sl48': 0.18604651162790697,
 'u36 sl48': 0.16279069767441856,
 'u48 sl48': 0.2558139534883721,
 'u64 sl48': 0.2790697674418605,
 'u72 sl48': 0.2093023255813954,
 'u128 sl48': 0.0,
 'u256 sl48': 0.16279069767441856,
 'u24 sl168': 0.2790697674418605,
 'u36 sl168': 0.2093023255813954,
 'u48 sl168': 0.2790697674418605,
 'u64 sl168': 0.2093023255813954,
 'u72 sl168': 0.2325581395348837,
 'u128 sl168': 0.2093023255813954,
 'u256 sl168': 0.18604651162790697}

## 2 layer

In [7]:
results = {}

for sequence_length in [12,24,48,24*7]:
    for units in [24,36,48,64,72,128,256]:
        site='prpa'
        epochs=300
        dropout=0
        patience=100
        layers=2
        verbose=0
        output = True
        plots = False
        filename = 'data/PRPA_load_cleaned_mjw.csv'
        shift_steps = 1
        dir = 'models'
        features = [  'Load (kW)',
                    'Day',
                    'Weekday',
                    'Hour',
                    'IMF1',                                
                    'IMF2',                                
                    'IMF3',
                    'IMF4',
                    'IMF5',
                    'IMF6',
                    'IMF7',
                    'IMF8',]
        targets = ['TargetsOH']
        train_split = 0.9
        afuncs={'lstm':'relu','dense':'relu','gru':'relu'}
        loss='binary_crossentropy'
        metrics=['accuracy']

        t = datetime.now()
        path_checkpoint = f'{dir}/{site}/{t.year}-{t.month:02}-{t.day:02}_'+\
                        f'{t.hour:02}-{t.minute:02}-{t.second:02}_lstm_{units}x{layers}x{shift_steps}.keras'

        df,dppd,np_days = ft.get_dat_v4(site,filename,emd=True,rename=True,start='2021-6-7',end='2022-8-10')                    

        df['LoadOH'] =      one_hot_of_peaks(df[['Load (kW)']])
        df['TargetsOH'] =   one_hot_of_peaks(df[['Load (kW)']]).shift(-shift_steps)
        df['PredNPOH'] =    one_hot_of_peaks(df[['Load (kW)']]).shift(np_days*dppd-shift_steps)
        df = df.dropna()
        
        # split
        num_data = len(df)
        num_train = int(train_split * num_data)
        df_train = df.iloc[:num_train,:]
        df_valid = df.iloc[num_train:,:]

        feature_scaler = MinMaxScaler()
        X_train = feature_scaler.fit_transform(df_train[features].values)
        X_valid = feature_scaler.fit_transform(df_valid[features].values)

        y_train = df_train.TargetsOH.values[:,np.newaxis]
        y_valid = df_valid.TargetsOH.values[:,np.newaxis]
        
        generator = ft.batch_generator( batch_size=32,
                                        sequence_length=sequence_length,
                                        num_x_signals=len(features),
                                        num_y_signals=len(targets),
                                        num_train=num_train,
                                        x_train_scaled=X_train,
                                        y_train_scaled=y_train)   
        
        X_batch, y_batch = next(generator)
        
        X_valid = X_valid[np.newaxis,:,:]
        y_valid = y_valid[np.newaxis,:,:] 

        model = Sequential()
        model.add( LSTM(  units=units,
                        return_sequences=True,
                        input_shape=(None, len(features),),
                        activation=afuncs['lstm']))
        model.add( LSTM(  units,
                        return_sequences=True,
                        activation=afuncs['lstm']))      
        model.add(Dense(len(targets), activation=afuncs['dense']))

        model.compile(loss=loss, optimizer='adam',metrics=metrics)
        model.summary()                  

        callback_checkpoint = ModelCheckpoint(  filepath=path_checkpoint,
                                                monitor='val_loss',
                                                verbose=verbose,
                                                save_weights_only=True,
                                                save_best_only=True)

        callback_early_stopping = EarlyStopping(  monitor='val_loss',
                                                patience=patience,
                                                verbose=verbose)

        callbacks = [ callback_early_stopping,
                    callback_checkpoint,]

        hx = model.fit(  x=generator,
                    epochs=100,
                    steps_per_epoch=100,
                    validation_data=(X_valid,y_valid),
                    callbacks=callbacks)        
        
        model.load_weights(path_checkpoint)
        y_valid_pred = model.predict(X_valid)
        
        y_valid_flat      = y_valid[:,:,0].flatten()
        y_valid_pred_flat = y_valid_pred[:,:,0].flatten()
        
        df_valid.loc[:,'y'] = y_valid_flat
        df_valid.loc[:,'y_pred'] = y_valid_pred_flat    
        
        results[f'u{units} sl{sequence_length}'] = accuracy_one_hot(df_valid['y'],one_hot_of_peaks(df_valid['y_pred']))
        

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, None, 24)          3552      
                                                                 
 lstm_29 (LSTM)              (None, None, 24)          4704      
                                                                 
 dense_56 (Dense)            (None, None, 1)           25        
                                                                 
Total params: 8,281
Trainable params: 8,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 3s 10ms/step - loss: 0.1717 - accuracy: 0.9593 - val_loss: 0.2143 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 1s 5ms/step - loss: 0.1626 - accuracy: 0.9577 - val_loss: 0.2399 - val_accuracy: 0.9583
Epoch 3/100
100/100 [=======

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, None, 36)          7056      
                                                                 
 lstm_31 (LSTM)              (None, None, 36)          10512     
                                                                 
 dense_57 (Dense)            (None, None, 1)           37        
                                                                 
Total params: 17,605
Trainable params: 17,605
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 2s 9ms/step - loss: 0.1703 - accuracy: 0.9580 - val_loss: 0.2059 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 1s 6ms/step - loss: 0.1611 - accuracy: 0.9580 - val_loss: nan - val_accuracy: 0.9583
Epoch 3/100
100/100 [=========

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_58"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_32 (LSTM)              (None, None, 48)          11712     
                                                                 
 lstm_33 (LSTM)              (None, None, 48)          18624     
                                                                 
 dense_58 (Dense)            (None, None, 1)           49        
                                                                 
Total params: 30,385
Trainable params: 30,385
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 2s 11ms/step - loss: 0.1717 - accuracy: 0.9579 - val_loss: 0.2194 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 1s 9ms/step - loss: 0.1521 - accuracy: 0.9586 - val_loss: 2.0379 - val_accuracy: 0.8660
Epoch 3/100
100/100 [=====

...

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_82"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_80 (LSTM)              (None, None, 128)         72192     
                                                                 
 lstm_81 (LSTM)              (None, None, 128)         131584    
                                                                 
 dense_82 (Dense)            (None, None, 1)           129       
                                                                 
Total params: 203,905
Trainable params: 203,905
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 16s 144ms/step - loss: 0.1573 - accuracy: 0.9584 - val_loss: 0.1301 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 14s 141ms/step - loss: 0.1894 - accuracy: 0.9581 - val_loss: 0.1332 - val_accuracy: 0.9573
Epoch 3/100
100/100

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


Model: "sequential_83"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_82 (LSTM)              (None, None, 256)         275456    
                                                                 
 lstm_83 (LSTM)              (None, None, 256)         525312    
                                                                 
 dense_83 (Dense)            (None, None, 1)           257       
                                                                 
Total params: 801,025
Trainable params: 801,025
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 33s 309ms/step - loss: 0.6385 - accuracy: 0.9583 - val_loss: nan - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 31s 313ms/step - loss: 0.6426 - accuracy: 0.9583 - val_loss: nan - val_accuracy: 0.9583
Epoch 3/100
100/100 [====

OSError: Unable to open file (unable to open file: name = 'models/prpa/2023-01-16_17-49-45_lstm_256x2x1.keras', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [8]:
results

{'u24 sl12': 0.2558139534883721,
 'u36 sl12': 0.3023255813953488,
 'u48 sl12': 0.2790697674418605,
 'u64 sl12': 0.2790697674418605,
 'u72 sl12': 0.0,
 'u128 sl12': 0.32558139534883723,
 'u256 sl12': 0.13953488372093026,
 'u24 sl24': 0.2325581395348837,
 'u36 sl24': 0.2325581395348837,
 'u48 sl24': 0.0,
 'u64 sl24': 0.2093023255813954,
 'u72 sl24': 0.2558139534883721,
 'u128 sl24': 0.2093023255813954,
 'u256 sl24': 0.18604651162790697,
 'u24 sl48': 0.16279069767441856,
 'u36 sl48': 0.18604651162790697,
 'u48 sl48': 0.06976744186046513,
 'u64 sl48': 0.2790697674418605,
 'u72 sl48': 0.18604651162790697,
 'u128 sl48': 0.2790697674418605,
 'u256 sl48': 0.18604651162790697,
 'u24 sl168': 0.2325581395348837,
 'u36 sl168': 0.3023255813953488,
 'u48 sl168': 0.2093023255813954,
 'u64 sl168': 0.2558139534883721,
 'u72 sl168': 0.32558139534883723,
 'u128 sl168': 0.39534883720930236}

u128 sl168

In [11]:
results = {}

for sequence_length in [24*7]:
    for units in [128]:
        site='prpa'
        epochs=100
        dropout=0
        patience=20
        layers=2
        verbose=0
        output = True
        plots = False
        filename = 'data/PRPA_load_cleaned_mjw.csv'
        shift_steps = 1
        dir = 'models'
        features = [  'Load (kW)',
                    'Day',
                    'Weekday',
                    'Hour',
                    'IMF1',                                
                    'IMF2',                                
                    'IMF3',
                    'IMF4',
                    'IMF5',
                    'IMF6',
                    'IMF7',
                    'IMF8',]
        targets = ['TargetsOH']
        train_split = 0.9
        afuncs={'lstm':'relu','dense':'relu','gru':'relu'}
        loss='binary_crossentropy'
        metrics=['accuracy']

        t = datetime.now()
        path_checkpoint = f'{dir}/{site}/{t.year}-{t.month:02}-{t.day:02}_'+\
                        f'{t.hour:02}-{t.minute:02}-{t.second:02}_lstm_{units}x{layers}x{shift_steps}.keras'

        df,dppd,np_days = ft.get_dat_v4(site,filename,emd=True,rename=True,start='2021-6-7',end='2022-8-10')                    

        df['LoadOH'] =      one_hot_of_peaks(df[['Load (kW)']])
        df['TargetsOH'] =   one_hot_of_peaks(df[['Load (kW)']]).shift(-shift_steps)
        df['PredNPOH'] =    one_hot_of_peaks(df[['Load (kW)']]).shift(np_days*dppd-shift_steps)
        df = df.dropna()
        
        # split
        num_data = len(df)
        num_train = int(train_split * num_data)
        df_train = df.iloc[:num_train,:]
        df_valid = df.iloc[num_train:,:]

        feature_scaler = MinMaxScaler()
        X_train = feature_scaler.fit_transform(df_train[features].values)
        X_valid = feature_scaler.fit_transform(df_valid[features].values)

        y_train = df_train.TargetsOH.values[:,np.newaxis]
        y_valid = df_valid.TargetsOH.values[:,np.newaxis]
        
        generator = ft.batch_generator( batch_size=32,
                                        sequence_length=sequence_length,
                                        num_x_signals=len(features),
                                        num_y_signals=len(targets),
                                        num_train=num_train,
                                        x_train_scaled=X_train,
                                        y_train_scaled=y_train)   
        
        X_batch, y_batch = next(generator)
        
        X_valid = X_valid[np.newaxis,:,:]
        y_valid = y_valid[np.newaxis,:,:] 

        model = Sequential()
        model.add( LSTM(  units=units,
                        return_sequences=True,
                        input_shape=(None, len(features),),
                        activation=afuncs['lstm']))
        model.add( LSTM(  units,
                        return_sequences=True,
                        activation=afuncs['lstm']))      
        model.add(Dense(len(targets), activation=afuncs['dense']))

        model.compile(loss=loss, optimizer='adam',metrics=metrics)
        model.summary()                  

        callback_checkpoint = ModelCheckpoint(  filepath=path_checkpoint,
                                                monitor='val_loss',
                                                verbose=verbose,
                                                save_weights_only=True,
                                                save_best_only=True)

        callback_early_stopping = EarlyStopping(  monitor='val_loss',
                                                patience=patience,
                                                verbose=verbose)

        callbacks = [ callback_early_stopping,
                    callback_checkpoint,]

        hx = model.fit(  x=generator,
                    epochs=100,
                    steps_per_epoch=100,
                    validation_data=(X_valid,y_valid),
                    callbacks=callbacks)        
        
        model.load_weights(path_checkpoint)
        y_valid_pred = model.predict(X_valid)
        
        y_valid_flat      = y_valid[:,:,0].flatten()
        y_valid_pred_flat = y_valid_pred[:,:,0].flatten()
        
        df_valid.loc[:,'y'] = y_valid_flat
        df_valid.loc[:,'y_pred'] = y_valid_pred_flat    
        
        results[f'u{units} sl{sequence_length}'] = accuracy_one_hot(df_valid['y'],one_hot_of_peaks(df_valid['y_pred']))
        
results

Model: "sequential_86"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_88 (LSTM)              (None, None, 128)         72192     
                                                                 
 lstm_89 (LSTM)              (None, None, 128)         131584    
                                                                 
 dense_86 (Dense)            (None, None, 1)           129       
                                                                 
Total params: 203,905
Trainable params: 203,905
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
100/100 [==============================] - 20s 184ms/step - loss: 0.1548 - accuracy: 0.9583 - val_loss: 0.1235 - val_accuracy: 0.9583
Epoch 2/100
100/100 [==============================] - 17s 173ms/step - loss: 0.1348 - accuracy: 0.9583 - val_loss: 0.1522 - val_accuracy: 0.9583
Epoch 3/100
100/100

/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mjw/miniconda3/envs/tf-scip2/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


{'u128 sl168': 0.18604651162790697}

(end)